Classification Network (using Resnet32) on CIFAR100

In [22]:
import torch 
import torchvision
import torchvision.transforms as transforms

### Data Load

In [23]:
# we build a transform to normalize images: Data normalization is an important step which ensures 
# each input parameter (pixel, in this case) has a similar data distribution. This makes convergence 
# faster while training the network.
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 4

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, 
                                         download=True, transform=transform)
# DataLoader. Combines a dataset and a sampler, and provides an iterable over the given dataset.
# batch_size = how many samples per batch to load
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


### Build the model

In [24]:
import torchvision.models as models

""" instantiate the net """
resnet34 = models.resnet18(pretrained=False)
#resnet34.fc.out_features = 10

""" move the model to the GPU """
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
resnet34.to(device)

cuda:0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Define the loss and the optimization technique

In [25]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet34.parameters(), lr=0.001, momentum=0.9)

### Training


In [ ]:
#train the network
for epoch in range(3):

  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):

    # get the inputs; data is a list of  [input,labels]
    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad() # Sets the gradients of all optimized torch.Tensor to zero.

    outputs = resnet34(inputs) # forward: assign weights to each edge in each layer
    loss = criterion(outputs,labels) # calculate the loss 
    loss.backward() # redesign the weights evaluating the performance of the network
    optimizer.step() # update parameters 

    running_loss += loss.item()
    if i % 2000 == 1999:    # print every 2000 mini-batches the average value of the loss accumulated in each batch
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

[1,  2000] loss: 3.901
[1,  4000] loss: 3.838
[1,  6000] loss: 3.806
[1,  8000] loss: 3.754
[1, 10000] loss: 3.685
[1, 12000] loss: 3.668
[2,  2000] loss: 3.539
[2,  4000] loss: 3.516
[2,  6000] loss: 3.444
[2,  8000] loss: 3.439
[2, 10000] loss: 3.397
[2, 12000] loss: 3.374
[3,  2000] loss: 3.245
[3,  4000] loss: 3.232
[3,  6000] loss: 3.194
[3,  8000] loss: 3.184
[3, 10000] loss: 3.169
[3, 12000] loss: 3.138


### Performance evaluation

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        outputs = resnet34(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))


Accuracy of the network on the 10000 test images: 23 %


In [32]:
# divided our dataset into sample of 10 classes each
# train the network on the first 10 classes
# evaluate the network on the first 10 classes
# train the network on the second 10 classes (adding 10 output layers)
# evaluate the network on the first 20 classes
iterations=10
test_set = [] #initialized here because we test over all the classes not only those one in which I train
# for epoch in range(3):
for i in range(iterations):
  #i = 0
  classes_current_iter = range(i*iterations, i*iterations+iterations)
  train_iter = [] 
  for j in range(len(trainset)):
    if(trainset[j][-1] in classes_current_iter):
      test_set.append(trainset[j]) 
      train_iter.append(trainset[j])

  train_loader = torch.utils.data.DataLoader(train_iter, shuffle = True, batch_size=batch_size, num_workers=2)
  valid_loader = torch.utils.data.DataLoader(test_set, shuffle = True, batch_size = batch_size, num_workers=2) 
  training(train_loader, i, resnet34) # Train the network with 10 classes at a time
  test(valid_loader, i, resnet34) # Test the network with all classes seen until this iteration

  # train loader contains at each iteration the new 10 classes used to evaluate the network, while valid loader contains all classes seen so far

Accuracy of the network on the 0 iteration: 62 %
Accuracy of the network on the 1 iteration: 18 %
Accuracy of the network on the 2 iteration: 16 %
Accuracy of the network on the 3 iteration: 10 %
Accuracy of the network on the 4 iteration: 9 %
Accuracy of the network on the 5 iteration: 7 %
Accuracy of the network on the 6 iteration: 7 %
Accuracy of the network on the 7 iteration: 6 %
Accuracy of the network on the 8 iteration: 4 %
Accuracy of the network on the 9 iteration: 5 %


In [26]:
# Each time we call training function we must pass a different trainloader, updated with the following 10 classes
def training(trainloader, iteration, resnet34):
  #if (iteration != 0):
    # add 10 output nodes to the network
    #num_classes = 10
    #addOutputNodes(resnet34, num_classes)

  #train the network
  for epoch in range(1):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):

      # get the inputs; data is a list of  [input,labels]
      inputs, labels = data[0].to(device), data[1].to(device)

      optimizer.zero_grad() # Sets the gradients of all optimized torch.Tensor to zero.

      outputs = resnet34(inputs) # forward: assign weights to each edge in each layer
      loss = criterion(outputs,labels) # calculate the loss 
      loss.backward() # redesign the weights evaluating the performance of the network
      optimizer.step() # update parameters 

      running_loss += loss.item()
      if i % 2000 == 1999:    # print every 2000 mini-batches the average value of the loss accumulated in each batch
        print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0

In [27]:
def addOutputNodes (network, num_classes):
  in_features = network.fc.in_features
  out_features = network.fc.out_features
  weight = network.fc.weight.data

  network.fc = nn.Linear(in_features, out_features + num_classes)
  network.fc.weight.data[:out_features] = weight

In [30]:
def test(testloader, iteration, resnet34):
  correct = 0
  total = 0
  # since we're not training, we don't need to calculate the gradients for our outputs
  with torch.no_grad():
      for data in testloader:
          images, labels = data[0].to(device), data[1].to(device)
          # calculate outputs by running images through the network
          outputs = resnet34(images)
          # the class with the highest energy is what we choose as prediction
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print(f'Accuracy of the network on the {iteration} iteration: %d %%' % (100 * correct / total))